In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

# Define paths
video_file = "/content/drive/MyDrive/OMSCS/Cichlids/clipped_file_15mins.mp4"
clips_dir = "/content/drive/MyDrive/OMSCS/Cichlids/clipped_clips/"

In [3]:
# Create the directory if it does not exist
if not os.path.exists(clips_dir):
    os.makedirs(clips_dir)

In [4]:
!pip install torch torchvision
!sudo apt update
!sudo apt install ffmpeg
!pip install av

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:5 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,392 kB]
Fetched 1,649 kB in 2s (880 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
45 packages can be upgraded

In [5]:
""" BEGIN CODE FROM https://raw.githubusercontent.com/Human-Augment-Analytics/CichlidBowerTracking/master/cichlid_bower_tracking/data_distillation/data/video_clipper.py"""

from typing import List
import math, os
from torchvision.io import read_video, write_video
import torch

class VideoClipper:
    def __init__(self, video_index: int, video_file: str, clips_dir: str, fps=30, fpc=300, debug=False):
        '''
        Initializes an instance of the VideoClipper class.

        Inputs:
            video_index: an int indicating the index of the video in a given project.
            video_file: the string local filepath to the video to be split up.
            clips_dir: the string local path to the directory where the clips will be stored.
            fps: an int indicating the framerate of the passed video (and therefore the clips); defaults to 30.
            fpc: an int indicating the number of "frames per clip"; defaults to 300, or 10 seconds (assuming fps=30).
        '''

        self.video_index = video_index
        self.video_file = video_file

        self.clips_dir = clips_dir
        self.fpc = fpc
        self.fps = fps

        self.debug = debug

        self.clip_count = 0

    def _save_clip(self, clip: torch.Tensor) -> None:
        '''
        Saves the passed stack of frames as a clip to the local file system.

        Inputs:
            clip: a Tensor containing frames from the passed video.
        '''

        filename = f'{str(self.clip_count + 1).zfill(9)}.mp4'
        filename = os.path.join(self.clips_dir, filename)

        video_codec = 'h264'

        if self.debug:
            print(f'\t...writing clip {filename}')

        write_video(filename, clip, self.fps, video_codec=video_codec)

    def run(self) -> None:
        '''
        Runs the VideoClipper on the passed video file.

        Inputs: None.
        '''

        prev = None
        start_idx = 0.0
        spc = self.fpc / self.fps

        clip = torch.randn(2)

        while torch.numel(clip) > 0:
            clip = read_video(filename=self.video_file, start_pts=start_idx, end_pts=start_idx + spc - (1/self.fps), pts_unit='sec', output_format='THWC')[0]

            if clip.shape[0] == 1:
                if prev is None:
                    prev = clip
                elif prev is not None and torch.equal(prev, clip):
                    break

            self._save_clip(clip=clip)
            self.clip_count += 1
            start_idx += spc

""" END CODE FROM https://raw.githubusercontent.com/Human-Augment-Analytics/CichlidBowerTracking/master/cichlid_bower_tracking/data_distillation/data/video_clipper.py"""

In [6]:
# Initialize and run the VideoClipper
clipper = VideoClipper(video_index=1, video_file=video_file, clips_dir=clips_dir, fps=30, fpc=300, debug=True)
clipper.run()

	...writing clip /content/drive/MyDrive/OMSCS/Cichlids/clipped_clips/000000001.mp4
	...writing clip /content/drive/MyDrive/OMSCS/Cichlids/clipped_clips/000000002.mp4
	...writing clip /content/drive/MyDrive/OMSCS/Cichlids/clipped_clips/000000003.mp4
	...writing clip /content/drive/MyDrive/OMSCS/Cichlids/clipped_clips/000000004.mp4
	...writing clip /content/drive/MyDrive/OMSCS/Cichlids/clipped_clips/000000005.mp4
	...writing clip /content/drive/MyDrive/OMSCS/Cichlids/clipped_clips/000000006.mp4
	...writing clip /content/drive/MyDrive/OMSCS/Cichlids/clipped_clips/000000007.mp4
	...writing clip /content/drive/MyDrive/OMSCS/Cichlids/clipped_clips/000000008.mp4
	...writing clip /content/drive/MyDrive/OMSCS/Cichlids/clipped_clips/000000009.mp4
	...writing clip /content/drive/MyDrive/OMSCS/Cichlids/clipped_clips/000000010.mp4
	...writing clip /content/drive/MyDrive/OMSCS/Cichlids/clipped_clips/000000011.mp4
	...writing clip /content/drive/MyDrive/OMSCS/Cichlids/clipped_clips/000000012.mp4
	...

In [ ]:
# Go through each of the 10 s clips one-by-one, and run the HMM logic on it

# ...enter code!

In [19]:
# Step 1: Reads the video and stores each frame in a list.

import cv2

def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()
    return frames

In [20]:
# Step 2: Compute the absolute difference between consecutive frames

def calculate_pixel_differences(frames):
    differences = []
    for i in range(1, len(frames)):
        diff = cv2.absdiff(frames[i], frames[i-1])
        differences.append(diff)
    return differences

In [21]:
# Step 3: Extract features from the pixel differences.
# The sum of differences is used as a simple feature.

import numpy as np

def extract_features(differences):
    features = []
    for diff in differences:
        # Example feature: sum of pixel differences
        feature = np.sum(diff)
        features.append(feature)
    return np.array(features).reshape(-1, 1)

In [12]:
!pip install hmmlearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 kB 1.5 MB/s eta 0:00:00


In [22]:
# Step 4: Trains a Gaussian HMM on the extracted features

from hmmlearn import hmm

def train_hmm(features):
    model = hmm.GaussianHMM(n_components=2, covariance_type="diag", n_iter=1000)
    model.fit(features)
    states = model.predict(features)
    return states

In [23]:
# Step 5: Identifies frames where the HMM state indicates a significant change

def detect_significant_changes(states):
    significant_changes = np.where(states == 1)[0]
    return significant_changes

In [27]:
# Step 6: Run the main function

def main(video_path):
    frames = extract_frames(video_path)
    pixel_differences = calculate_pixel_differences(frames)
    features = extract_features(pixel_differences)
    states = train_hmm(features)
    significant_changes = detect_significant_changes(states)
    return significant_changes

In [30]:
# Directory containing the clips
clips_dir = "/content/drive/MyDrive/OMSCS/Cichlids/clipped_clips"

# Process each clip
for i in range(1, 93):
    clip_number = str(i).zfill(9)
    video_path = os.path.join(clips_dir, f"{clip_number}.mp4")
    if os.path.exists(video_path):
        significant_changes = main(video_path)
        print(f"Number of significant changes at clip {i}: {len(significant_changes)}")
    else:
        print(f"Clip {i} does not exist.")

Number of significant changes at clip 1: 1
Number of significant changes at clip 2: 1
Number of significant changes at clip 3: 1
Number of significant changes at clip 4: 1
Number of significant changes at clip 5: 1
Number of significant changes at clip 6: 1
Number of significant changes at clip 7: 1
Number of significant changes at clip 8: 1
Number of significant changes at clip 9: 1
Number of significant changes at clip 10: 1
Number of significant changes at clip 11: 1
Number of significant changes at clip 12: 1
Number of significant changes at clip 13: 1
Number of significant changes at clip 14: 298
Number of significant changes at clip 15: 1
Number of significant changes at clip 16: 1
Number of significant changes at clip 17: 1
Number of significant changes at clip 18: 1
Number of significant changes at clip 19: 1
Number of significant changes at clip 20: 1
Number of significant changes at clip 21: 1
Number of significant changes at clip 22: 1
Number of significant changes at clip 2